<img src="figures/ampel.png" width="200">

### AMPEL and the Vera C. Rubin Observatory

The Vera Rubin Observatory, and the LSST survey, will provide a legacy collection of real-time data. Considering the potential long term impact of any transient programs, the AMPEL analysis platform was developed to 
host complex science program with provenance requirements matching those of the observatory. In essence, this means the creation of _scientific analysis schema_ which detail all scientific/algorithmic choices being made. This schema can be distributed with publications, and consistently applied to simulated, archived and real-time datasets.

##### This notebook : Real-time multi-messenger programs

This notebook presents options for real-time multi-messenger channels using the AMPEL framework. It is closely aligned with `Tutorial 3` from this repository.

### Multi-messenger analysis based on a primary stream

The simplest way to do a real-time "multi-messenger" analysis is to compare a "driving" stream to some external catalog containing recent MM events. This is conceptually similar to querying a compiled catalog offline.

We here demo the use of an analysis T2 unit to compare selected optical transients with a catalog of recent multi-messenger events, ```T2MultiMessMatch```. 

In [ ]:
import os
%load_ext ampel_quick_import
%qi DevAmpelContext AmpelLogger T2Processor T3Processor ChannelModel AlertProcessor TarAlertLoader ChannelModel AbsAlertFilter T2MultiMessMatch

In [ ]:
T2MultiMessMatch??

A unit - `T2MultiMessMatch` - retrieves a recent list of potential multi-messenger events, together with information of their spatial, temporal and energetics properties. It will then return a combined score of how closely these match the properties of the optical transients. A set of scaling factors determine the relative weight between these dimensions. A real unit would read this information from a local mirror of e.g. LIGO/VIRGO or IceCube events, while we here use an invented event.

Path to unit: `Ampel-contrib-sample/ampel/contrib/sample/t2/T2MultiMessMatch.py`

The final step of this notebook is the selection of events we consider "good" matches, based on having a small p-value.

First, we set up our operation context from a configuration file.
`db_prefix` sets the DB name to use.

In [ ]:

AMPEL_CONF = "/opt/env/etc/ampel_config.yml"
ALERT_ARCHIVE = '../sample_data/ztfpub_200917_pruned.tar.gz'

ctx = DevAmpelContext.load(
    config_file_path = AMPEL_CONF,
    db_prefix = "AmpelTutorial",
    purge_db = True,
)

In [ ]:
# Register our channel (scientific program)
ctx.add_channel(
    name="demo_SN09if",
    access=['ZTF', 'ZTF_PUB']
)

A channel can specify which streams to read, how these should be combined and what units should be run on each data combination.
These are provided as directives to the AlertProcessor, which filters incoming alerts and triggers further calculations for transients that pass the filter stage.

In [ ]:

ap = AlertProcessor(
    context = ctx,
    process_name = "ipython_notebook_test",
    supplier = "ZiAlertSupplier",
    loader = {"unit": "TarAlertLoader", "config": {"file_path": ALERT_ARCHIVE}},
    iter_max = 200,
    log_profile = "debug",
    directives = [
        {
            "channel": "demo_SN09if",
            # The filter controls which datapoints enter the analysis.
            # Here we just apply some basic quality cuts.
            "filter": {
                "unit": "SimpleDecentFilterCopy",
                "config": ctx.add_config_id({
                    'min_rb':0.3,
                    'min_ndet':7,
                    'min_tspan':10,
                    'max_tspan' : 200,
                    'min_gal_lat':15,
                })
            },
            "stock_update": "ZiStockIngester",
            # whenever a new datapoint is added...
            't0_add': {
                "ingester": "ZiAlertContentIngester",
                # update states (light curves)
                "t1_combine": [
                    {
                        "ingester": "PhotoCompoundIngester",
                        "config": {"combiner": "ZiT1Combiner"},
                        # and calculate secondary quantities
                        "t2_compute": {
                            "ingester": "PhotoT2Ingester",
                            "config": {"tags": ["ZTF"]},
                            "units": [
                                # Light-curve ranking: score fit against salt2
                                {'unit': 'T2SNcosmoComp',
                                 'config': ctx.add_config_id({
                                    'target_model_name': 'v19-2009ip-corr', # alternate model
                                    'base_model_name'  : 'salt2',           # null model
                                    'chi2dof_cut'      : 2.,                # maximum reduced chi^2 allowed for alternate model
                                    'chicomp_scaling'  : 0.5,               # conversion from delta-chi^2 to score
                                 })
                                },
                                # Multi-messenger match: score light curve against target list
                                {'unit': 'T2MultiMessMatch',
                                 'config': ctx.add_config_id({
                                    'temporal_pull_scaling' : 1,         # Neutral - we do not know when Neutrinos are emitted
                                    'spatial_pull_scaling'  : 3.,        # Reasonably sure regarding location
                                    'energy_pull_scaling'   : 0.001,     # Little constraint on energy, deweight this
                                    'match_where'           : 'latest',  # latest, first or mean
                                  })
                                },
                            ]
                        }
                    }
                ],
            }
        }
    ]
)

In [ ]:
ap.run()

In [ ]:
for collection in "stock", "t0", "t1", "t2":
    print(f"{collection}: {ctx.db.get_collection(collection).estimated_document_count()}")


In [ ]:
t2p = T2Processor(context=ctx, process_name="T2Processor_test", log_profile="verbose")

In [ ]:
t2p.run()

In deciding which targets to follow we wish to also make use of the match criteria. Can be done as follows:

In [ ]:
t3 = T3Processor(
    context=ctx,
    process_name = "T3Processor_test",
    log_profile = "default", # debug
    channel = "demo_SN09if",
    directives = [ {
        "select": {
            "unit": "T3FilteringStockSelector",
            "config": {
                't2_filter': { 
                    'all_of': [                
                        {
                            'unit': 'T2SNcosmoComp',
                            'match': {'target_match': True}
                        }, 
                        {
                            'unit': 'T2MultiMessMatch',
                            'match': {'best_match': {"$lt":1} }
                        },                         
                    ]
            } }
        },
        "load": {
            "unit": "T3SimpleDataLoader",
            "config": {
                "directives": ["TRANSIENT", "DATAPOINT", "COMPOUND", "T2RECORD"],
            }

        },
        "run": {
            "unit": "T3UnitRunner",
            "config": {
                "directives": [
                      {
                            "execute": [
                                {
                                    "unit": "T3HelloWorld",
                                    "config": {
                                        't2info_from' : ['T2SNcosmoComp', 'T2MultiMessMatch']
                                    },
                                },
                            ]

                      }
                ]
            }
        }
    } ]
)

In [ ]:
t3.run()

Looks like ZTF20abyfpze is our target! This analysis schema for real-time analysis can now be integrated into a live AMPEL instance. 

### Multi-messenger analysis based on hybrid states

A _state_ collects real-time information associated to one transient event and available to one observer at some time. A _hybrid_ state is the product of the processing of heterogenous data-streams, where datapoints can have fundamentally different properties (e.g. Vega magnitude for an optical alert and mass for a gravitational wave alert). 

AMPEL allows for the creation of such hybrid states through connecting to two (or more) streams. This allows a full analysis that takes incorporates all available data. A T1 _combine_ unit associate datapoints from different origin into potential states (which are available for analysis downstream in the T2 layer). 

The following sample analysis schema shows a setup where a public LSST stream and a private HopSkotch stream (e.g. IceCube neutrino events below the public alert threshold) are ingested.

Scientific program choices are made in the filters as well as in the `PhotoNeutrinoCompoundIngester`.

```
...
controller:
  - unit: LSSTAlertStreamClient             # Client for reader LSST alert stream
      processor:
        unit: AlertProcessor
          directives:
            filter:
              unit: OpticalDataFilter       # Filter LSST data based on minimal usage criteria
            t0_add:
              unit: ZiAlertContentIngester  # Ingest optical data
  - unit: HopskotchStreamClient             # Wrapper around HopSkotch client
      config:
        server: server_name                 
        topic: my_private_topic             # Topic for private neutrino alert distribution
        auth: SECRET
      processor:
        unit: AlertProcessor
          directives:
            filter:
              unit: NeutrinoDataFilter      # Minimal significance criteria for accepting neutrino alerts
            t0_add:
              unit: HsAlertContentIngester  # Ingest content of stream
t1_combine:
  unit: PhotoNeutrinoCompoundIngester       # Unit w. recipe for associating data
    config:
      time_tolerance: 42                    # Sample parameter: how much can the emission times vary
...
```